# Sharpe

### About Sharpe
The Sharpe Ratio is a method to evaluate an asset's return in relation to it's risk. It is doing so by dividing the "exess return" by it's standard deviation. The "exess return" is the difference of the average return to the one of a "risk free" asses (such as US Government Bonds).

<img src="https://www.myaccountingcourse.com/wp-content/uploads/2017/10/sharpe-ratio-formula.jpg" width="750">

The below chart illustrates an example.

<img src="http://www.marottaonmoney.com/wp-content/uploads/2017/09/chile-sharpe.jpg" width="750">


### Approach
1. Import libraries
2. Data and data preparation